In [22]:
from glob import glob

In [23]:
stripfiles = sorted(glob('meshes/*.obj'))
stripfiles = [fl for fl in stripfiles if 't1' not in fl]
print('    ', len(stripfiles))
print(stripfiles[0])
print(stripfiles[-1])

     139
meshes\frame46000_strip_s16_1226.obj
meshes\frame46000_strip_s60_730.obj


In [24]:
def indadd(istr, amount):
    parts = [str(int(val) + amount) for val in istr.split()[1:]]
    return 'f ' + ' '.join(parts)

print(indadd('f 1 2 3 ', 10))

def norm(ww, vert):
    coords = [float(val) / float(ww.split('# w ')[1]) for val in vert.split()[1:]]
    return 'v ' + ' '.join([str(val) for val in coords])

print(norm(*('# w 2', 'v 2 2 2')))

f 11 12 13
v 1.0 1.0 1.0


In [25]:
verts = []
inds = []
verts_normed = []
verts_og = []

voffset = 0
FROM =0
UPTO = 150
for fname in stripfiles[FROM:UPTO]:
    with open(fname) as fl:
        raw = fl.read()
    lines = raw.split('\n')
    lines = [ll for ll in lines if ll]
    vs = [ll for ll in lines if ll[0] == 'v']
    
    ws = [ll for ll in lines if ll[:3] == '# w']
    os = [ll.replace('# o', 'v') for ll in lines if ll[:3] == '# o']
    fs = [ll for ll in lines if ll[0] == 'f']
    print('%s: %d verts, %d faces, %d ws' % (fname, len(vs), len(fs), len(ws)))
    
    verts_normed += [norm(*pair) for pair in zip(ws, vs)]
    verts_og += os
    verts += ['# ' + fname]
    verts += vs
    inds += [indadd(fdef, voffset) for fdef in fs]
    voffset += len(vs)

print('Total:  %d verts,  %d faces' % (len(verts), len(inds)))

meshes\frame46000_strip_s16_1226.obj: 378 verts, 1224 faces, 378 ws
meshes\frame46000_strip_s16_555.obj: 251 verts, 553 faces, 251 ws
meshes\frame46000_strip_s20_1118.obj: 407 verts, 1116 faces, 407 ws
meshes\frame46000_strip_s20_279.obj: 102 verts, 277 faces, 102 ws
meshes\frame46000_strip_s20_326.obj: 116 verts, 324 faces, 116 ws
meshes\frame46000_strip_s20_445.obj: 196 verts, 443 faces, 196 ws
meshes\frame46000_strip_s20_455.obj: 156 verts, 453 faces, 156 ws
meshes\frame46000_strip_s20_666.obj: 268 verts, 664 faces, 268 ws
meshes\frame46000_strip_s20_704.obj: 288 verts, 702 faces, 288 ws
meshes\frame46000_strip_s20_766.obj: 253 verts, 764 faces, 253 ws
meshes\frame46000_strip_s28_1016.obj: 680 verts, 1014 faces, 680 ws
meshes\frame46000_strip_s28_1024.obj: 656 verts, 1022 faces, 656 ws
meshes\frame46000_strip_s28_1028.obj: 688 verts, 1026 faces, 688 ws
meshes\frame46000_strip_s28_1076.obj: 720 verts, 1074 faces, 720 ws
meshes\frame46000_strip_s28_1088.obj: 728 verts, 1086 faces, 728

meshes\frame46000_strip_s60_2608.obj: 913 verts, 2606 faces, 913 ws
meshes\frame46000_strip_s60_4651.obj: 2274 verts, 4649 faces, 2274 ws
meshes\frame46000_strip_s60_474.obj: 288 verts, 472 faces, 288 ws
meshes\frame46000_strip_s60_516.obj: 187 verts, 514 faces, 187 ws
meshes\frame46000_strip_s60_730.obj: 322 verts, 728 faces, 322 ws
Total:  108228 verts,  171993 faces


In [26]:
with open('dump.obj', 'w') as fl:
    fl.write('\n'.join(verts))
    fl.write('\n')
    fl.write('\n'.join(inds))

In [27]:
with open('dump_og.obj', 'w') as fl:
    fl.write('\n'.join(verts_og))
    fl.write('\n')
    fl.write('\n'.join(inds))